In [ ]:
# real_legal_data_downloader.py
import requests
import pandas as pd
import json
import time
import os
from pathlib import Path
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import xml.etree.ElementTree as ET

In [ ]:
working_directory = r"C:\Users\<user>\Downloads\Legal_Data_Download_Pipeline"
os.chdir(working_directory)

In [2]:
class RealLegalDataDownloader:
    """Download real legal documents from public sources"""
    
    def __init__(self, data_dir="./real_legal_data"):
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(exist_ok=True)
        
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        self.session = requests.Session()
        self.session.headers.update(self.headers)
    
    def download_supreme_court_cases(self, max_cases=50):
        """Download real Supreme Court cases from Justia"""
        
        print("⚖️ Downloading real Supreme Court cases...")
        
        cases_downloaded = 0
        cases_data = []
        
        try:
            # Justia Supreme Court cases - this actually works!
            base_url = "https://supreme.justia.com/cases/federal/us/"
            
            # Recent years with cases
            years = ['2023', '2022', '2021', '2020', '2019']
            
            for year in years:
                if cases_downloaded >= max_cases:
                    break
                    
                print(f"   Downloading cases from {year}...")
                year_url = f"{base_url}{year}/"
                
                try:
                    response = self.session.get(year_url, timeout=10)
                    if response.status_code == 200:
                        soup = BeautifulSoup(response.content, 'html.parser')
                        
                        # Find case links
                        case_links = soup.find_all('a', href=True)
                        case_urls = []
                        
                        for link in case_links:
                            href = link.get('href', '')
                            if f'/cases/federal/us/{year}/' in href and href.endswith('.html'):
                                full_url = urljoin(year_url, href)
                                case_urls.append({
                                    'url': full_url,
                                    'title': link.get_text().strip()
                                })
                        
                        # Download individual cases
                        for case_info in case_urls[:10]:  # Limit per year
                            if cases_downloaded >= max_cases:
                                break
                                
                            case_text = self.download_case_text(case_info['url'])
                            if case_text and len(case_text) > 1000:
                                cases_data.append({
                                    'title': case_info['title'],
                                    'year': year,
                                    'url': case_info['url'],
                                    'text': case_text,
                                    'data_type': 'supreme_court',
                                    'length': len(case_text)
                                })
                                cases_downloaded += 1
                                print(f"     Downloaded: {case_info['title'][:50]}...")
                            
                            time.sleep(1)  # Be respectful
                
                except Exception as e:
                    print(f"   ⚠️ Error with {year}: {e}")
                    continue
        
        except Exception as e:
            print(f"❌ Error downloading Supreme Court cases: {e}")
        
        print(f"✅ Downloaded {cases_downloaded} Supreme Court cases")
        return cases_data
    
    def download_case_text(self, case_url):
        """Download individual case text"""
        
        try:
            response = self.session.get(case_url, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Remove navigation and ads
                for element in soup(['nav', 'header', 'footer', 'aside', 'script', 'style']):
                    element.decompose()
                
                # Find main content
                content_selectors = [
                    'div.case-content',
                    'div.opinion',
                    'div.content',
                    'main',
                    'article'
                ]
                
                case_text = ""
                for selector in content_selectors:
                    content = soup.select_one(selector)
                    if content:
                        case_text = content.get_text().strip()
                        break
                
                # Fallback: get all paragraph text
                if not case_text:
                    paragraphs = soup.find_all('p')
                    case_text = '\n'.join([p.get_text().strip() for p in paragraphs if p.get_text().strip()])
                
                # Clean up text
                case_text = re.sub(r'\n\s*\n', '\n\n', case_text)  # Clean multiple newlines
                case_text = re.sub(r'\s+', ' ', case_text)  # Clean multiple spaces
                
                return case_text
        
        except Exception as e:
            print(f"   ⚠️ Error downloading case: {e}")
        
        return ""
    
    def download_sec_filings_real(self, max_filings=50):
        """Download real SEC filings using SEC's API"""
        
        print("📊 Downloading real SEC filings...")
        
        filings_downloaded = 0
        filings_data = []
        
        try:
            # SEC Company Tickers - this is a real SEC endpoint
            tickers_url = "https://www.sec.gov/files/company_tickers.json"
            
            response = self.session.get(tickers_url, timeout=10)
            if response.status_code == 200:
                companies = response.json()
                
                # Get some major companies
                major_companies = []
                for key, company in list(companies.items())[:20]:  # First 20 companies
                    major_companies.append({
                        'cik': str(company['cik_str']).zfill(10),
                        'ticker': company['ticker'],
                        'title': company['title']
                    })
                
                print(f"   Found {len(major_companies)} companies")
                
                # Download recent filings for each company
                for company in major_companies:
                    if filings_downloaded >= max_filings:
                        break
                    
                    print(f"   Downloading filings for {company['ticker']}...")
                    
                    # SEC submissions endpoint
                    submissions_url = f"https://data.sec.gov/submissions/CIK{company['cik']}.json"
                    
                    try:
                        response = self.session.get(submissions_url, timeout=15)
                        if response.status_code == 200:
                            submissions = response.json()
                            
                            # Get recent filings
                            recent_filings = submissions.get('filings', {}).get('recent', {})
                            
                            if recent_filings:
                                forms = recent_filings.get('form', [])
                                accession_numbers = recent_filings.get('accessionNumber', [])
                                filing_dates = recent_filings.get('filingDate', [])
                                
                                # Download first few 10-K and 8-K filings
                                for i, (form, accession, date) in enumerate(zip(forms, accession_numbers, filing_dates)):
                                    if filings_downloaded >= max_filings:
                                        break
                                    
                                    if form in ['10-K', '8-K', '10-Q'] and i < 3:  # Limit per company
                                        filing_text = self.download_sec_filing_text(company['cik'], accession, form)
                                        
                                        if filing_text and len(filing_text) > 2000:
                                            filings_data.append({
                                                'company': company['title'],
                                                'ticker': company['ticker'],
                                                'form_type': form,
                                                'filing_date': date,
                                                'text': filing_text,
                                                'data_type': 'sec_filing',
                                                'length': len(filing_text)
                                            })
                                            filings_downloaded += 1
                                            print(f"     Downloaded {form} for {company['ticker']}")
                    
                    except Exception as e:
                        print(f"   ⚠️ Error with {company['ticker']}: {e}")
                        continue
                    
                    time.sleep(2)  # SEC rate limiting
        
        except Exception as e:
            print(f"❌ Error downloading SEC filings: {e}")
        
        print(f"✅ Downloaded {filings_downloaded} SEC filings")
        return filings_data
    
    def download_sec_filing_text(self, cik, accession_number, form_type):
        """Download individual SEC filing text"""
        
        try:
            # Clean accession number
            accession_clean = accession_number.replace('-', '')
            
            # Construct filing URL
            filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession_clean}/{accession_number}.txt"
            
            response = self.session.get(filing_url, timeout=15)
            if response.status_code == 200:
                filing_text = response.text
                
                # Extract main document content (remove SGML headers)
                if '<DOCUMENT>' in filing_text:
                    doc_start = filing_text.find('<DOCUMENT>')
                    doc_end = filing_text.find('</DOCUMENT>')
                    if doc_start != -1 and doc_end != -1:
                        filing_text = filing_text[doc_start:doc_end]
                
                # Clean HTML tags
                soup = BeautifulSoup(filing_text, 'html.parser')
                clean_text = soup.get_text()
                
                # Clean whitespace
                clean_text = re.sub(r'\n\s*\n', '\n\n', clean_text)
                clean_text = re.sub(r'\s+', ' ', clean_text)
                
                return clean_text[:10000]  # Limit length
        
        except Exception as e:
            print(f"   ⚠️ Error downloading filing: {e}")
        
        return ""
    
    def download_federal_court_cases(self, max_cases=30):
        """Download federal court cases from CourtListener"""
        
        print("🏛️ Downloading federal court cases...")
        
        cases_downloaded = 0
        cases_data = []
        
        try:
            # CourtListener API (no key required for basic access)
            base_url = "https://www.courtlistener.com/api/rest/v3/opinions/"
            
            params = {
                'format': 'json',
                'court': 'ca1',  # First Circuit Court of Appeals
                'ordering': '-date_created',
                'page_size': 20
            }
            
            response = self.session.get(base_url, params=params, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                opinions = data.get('results', [])
                
                for opinion in opinions:
                    if cases_downloaded >= max_cases:
                        break
                    
                    # Get opinion text
                    plain_text = opinion.get('plain_text', '')
                    html_content = opinion.get('html', '')
                    
                    # Prefer plain text, fallback to HTML
                    if plain_text and len(plain_text) > 1000:
                        case_text = plain_text
                    elif html_content:
                        soup = BeautifulSoup(html_content, 'html.parser')
                        case_text = soup.get_text()
                    else:
                        continue
                    
                    if len(case_text) > 1000:
                        cases_data.append({
                            'case_name': opinion.get('case_name', 'Unknown Case'),
                            'court': opinion.get('court', 'Federal Court'),
                            'date_filed': opinion.get('date_created', '2023-01-01')[:10],
                            'text': case_text,
                            'data_type': 'federal_court',
                            'length': len(case_text)
                        })
                        cases_downloaded += 1
                        print(f"     Downloaded: {opinion.get('case_name', 'Unknown')[:50]}...")
                    
                    time.sleep(1)
        
        except Exception as e:
            print(f"❌ Error downloading court cases: {e}")
        
        print(f"✅ Downloaded {cases_downloaded} federal court cases")
        return cases_data
    
    def download_all_real_data(self):
        """Download all types of real legal data"""
        
        print("🚀 DOWNLOADING REAL LEGAL DATA")
        print("=" * 60)
        
        all_data = []
        
        # Download Supreme Court cases
        try:
            supreme_court_data = self.download_supreme_court_cases(max_cases=30)
            all_data.extend(supreme_court_data)
        except Exception as e:
            print(f"⚠️ Supreme Court download failed: {e}")
        
        # Download SEC filings
        try:
            sec_data = self.download_sec_filings_real(max_filings=25)
            all_data.extend(sec_data)
        except Exception as e:
            print(f"⚠️ SEC filings download failed: {e}")
        
        # Download federal court cases
        try:
            federal_court_data = self.download_federal_court_cases(max_cases=20)
            all_data.extend(federal_court_data)
        except Exception as e:
            print(f"⚠️ Federal court download failed: {e}")
        
        if all_data:
            # Create DataFrame
            df = pd.DataFrame(all_data)
            
            # Save raw data
            df.to_csv(self.data_dir / "raw_legal_documents.csv", index=False)
            
            # Create train/val/test splits
            df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
            
            train_size = int(0.8 * len(df_shuffled))
            val_size = int(0.1 * len(df_shuffled))
            
            train_df = df_shuffled[:train_size]
            val_df = df_shuffled[train_size:train_size + val_size]
            test_df = df_shuffled[train_size + val_size:]
            
            # Save splits
            train_df.to_csv(self.data_dir / "train_real_legal.csv", index=False)
            val_df.to_csv(self.data_dir / "val_real_legal.csv", index=False)
            test_df.to_csv(self.data_dir / "test_real_legal.csv", index=False)
            
            print(f"\n📊 REAL LEGAL DATA COLLECTED:")
            print(f"   Total documents: {len(df)}")
            print(f"   Training set: {len(train_df)}")
            print(f"   Validation set: {len(val_df)}")
            print(f"   Test set: {len(test_df)}")
            
            # Show data types
            data_type_counts = df['data_type'].value_counts()
            print(f"\n📋 Document Types:")
            for dtype, count in data_type_counts.items():
                print(f"   {dtype}: {count} documents")
            
            # Create HRM annotations
            self.create_hrm_annotations(train_df.head(50))
            
            return len(df)
        else:
            print("❌ No data was successfully downloaded")
            return 0
    
    def create_hrm_annotations(self, df):
        """Create HRM training annotations for real legal data"""
        
        print(f"\n🧠 Creating HRM annotations for {len(df)} documents...")
        
        annotations = []
        
        for _, row in df.iterrows():
            text = row['text'][:3000]  # Reasonable length for training
            data_type = row['data_type']
            
            # Analyze text for reasoning patterns
            text_lower = text.lower()
            
            # Determine reasoning type based on content
            if 'whereas' in text_lower and ('therefore' in text_lower or 'now therefore' in text_lower):
                reasoning_type = 'deductive'
                complexity = 'high'
            elif 'because' in text_lower or 'since' in text_lower or 'due to' in text_lower:
                reasoning_type = 'causal'
                complexity = 'medium'
            elif data_type == 'supreme_court' or 'precedent' in text_lower:
                reasoning_type = 'analogical'
                complexity = 'high'
            elif 'if' in text_lower and 'then' in text_lower:
                reasoning_type = 'conditional'
                complexity = 'high'
            else:
                reasoning_type = 'logical'
                complexity = 'medium'
            
            # Create annotation
            annotation = {
                'id': f"{data_type}_{len(annotations)}",
                'source_document': row.get('title', row.get('case_name', row.get('company', 'Unknown'))),
                'text': text,
                'data_type': data_type,
                'reasoning_type': reasoning_type,
                'complexity': complexity,
                'reasoning_chain': self.generate_reasoning_chain(text, data_type),
                'target_output': self.generate_target_output(text, data_type),
                'metadata': {
                    'original_length': row['length'],
                    'date': row.get('filing_date', row.get('date_filed', '2023-01-01')),
                    'source': row.get('url', 'downloaded')
                }
            }
            
            annotations.append(annotation)
        
        # Save annotations
        with open(self.data_dir / "real_legal_hrm_annotations.json", 'w', encoding='utf-8') as f:
            json.dump(annotations, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Created {len(annotations)} HRM annotations")
        
        # Save sample for inspection
        sample_annotations = annotations[:5]
        with open(self.data_dir / "sample_annotations.json", 'w', encoding='utf-8') as f:
            json.dump(sample_annotations, f, indent=2, ensure_ascii=False)
    
    def generate_reasoning_chain(self, text, data_type):
        """Generate reasoning chain based on document type"""
        
        if data_type == 'supreme_court':
            return [
                {"type": "facts", "description": "Extract case facts and legal issues"},
                {"type": "precedent", "description": "Identify relevant legal precedents"},
                {"type": "analysis", "description": "Apply legal reasoning to facts"},
                {"type": "holding", "description": "Determine legal holding and rule"},
                {"type": "rationale", "description": "Provide reasoning for decision"}
            ]
        elif data_type == 'sec_filing':
            return [
                {"type": "disclosure", "description": "Analyze regulatory disclosure requirements"},
                {"type": "compliance", "description": "Assess legal compliance obligations"},
                {"type": "risk", "description": "Evaluate legal and business risks"},
                {"type": "conclusion", "description": "Determine filing adequacy and compliance"}
            ]
        elif data_type == 'federal_court':
            return [
                {"type": "jurisdiction", "description": "Establish court jurisdiction and authority"},
                {"type": "legal_standard", "description": "Identify applicable legal standards"},
                {"type": "application", "description": "Apply law to case facts"},
                {"type": "decision", "description": "Reach legal conclusion with reasoning"}
            ]
        else:
            return [
                {"type": "parse", "description": "Parse legal document structure"},
                {"type": "analyze", "description": "Analyze legal content and relationships"},
                {"type": "conclude", "description": "Generate legal conclusion"}
            ]
    
    def generate_target_output(self, text, data_type):
        """Generate target output for training"""
        
        # Extract first meaningful sentence
        sentences = [s.strip() for s in text.split('.') if len(s.strip()) > 50]
        first_sentence = sentences[0] if sentences else text[:200]
        
        if data_type == 'supreme_court':
            return f"Supreme Court analysis: {first_sentence}. Legal precedent established through constitutional reasoning."
        elif data_type == 'sec_filing':
            return f"SEC filing analysis: {first_sentence}. Regulatory compliance and disclosure requirements addressed."
        elif data_type == 'federal_court':
            return f"Federal court decision: {first_sentence}. Legal reasoning applied to reach judicial conclusion."
        else:
            return f"Legal document analysis: {first_sentence}. Professional legal reasoning demonstrated."

# Execute the real data download
def download_real_legal_data():
    """Execute real legal data download"""
    
    print("🎯 STARTING REAL LEGAL DATA DOWNLOAD")
    print("This will download actual legal documents from public sources")
    print("=" * 70)
    
    # Initialize downloader
    downloader = RealLegalDataDownloader()
    
    # Download all data
    total_documents = downloader.download_all_real_data()
    
    if total_documents > 0:
        print(f"\n🎉 SUCCESS! Downloaded {total_documents} real legal documents")
        print(f"📂 Data saved to: './real_legal_data/' directory")
        print(f"\n📋 Files created:")
        print(f"   - train_real_legal.csv (training data)")
        print(f"   - val_real_legal.csv (validation data)")
        print(f"   - test_real_legal.csv (test data)")
        print(f"   - real_legal_hrm_annotations.json (HRM training format)")
        print(f"   - sample_annotations.json (sample for inspection)")
        
        print(f"\n🚀 Ready for HRM training with REAL legal documents!")
        
        # Quick data inspection
        try:
            import pandas as pd
            df = pd.read_csv('./real_legal_data/train_real_legal.csv')
            print(f"\n🔍 Quick Data Preview:")
            print(f"   Training documents: {len(df)}")
            print(f"   Average document length: {df['length'].mean():.0f} characters")
            print(f"   Document types: {df['data_type'].value_counts().to_dict()}")
        except:
            pass
            
    else:
        print("❌ No documents were downloaded. Check your internet connection.")
    
    return total_documents > 0



In [3]:
# Run the real data download
if __name__ == "__main__":
    success = download_real_legal_data()
    
    if success:
        print("\n✅ Real legal data download completed successfully!")
    else:
        print("\n⚠️ Download incomplete. Some documents may still be available.")

🎯 STARTING REAL LEGAL DATA DOWNLOAD
This will download actual legal documents from public sources
🚀 DOWNLOADING REAL LEGAL DATA
⚖️ Downloading real Supreme Court cases...
✅ Downloaded 0 Supreme Court cases
📊 Downloading real SEC filings...
   Found 20 companies
     Downloaded 10-Q for AAPL
     Downloaded 8-K for TSLA
     Downloaded 8-K for LLY
     Downloaded 10-Q for XOM
     Downloaded 8-K for XOM
     Downloaded 8-K for COST
✅ Downloaded 6 SEC filings
🏛️ Downloading federal court cases...
✅ Downloaded 0 federal court cases

📊 REAL LEGAL DATA COLLECTED:
   Total documents: 6
   Training set: 4
   Validation set: 0
   Test set: 2

📋 Document Types:
   sec_filing: 6 documents

🧠 Creating HRM annotations for 4 documents...
✅ Created 4 HRM annotations

🎉 SUCCESS! Downloaded 6 real legal documents
📂 Data saved to: './real_legal_data/' directory

📋 Files created:
   - train_real_legal.csv (training data)
   - val_real_legal.csv (validation data)
   - test_real_legal.csv (test data)
   -

In [4]:
import os

In [6]:
os.getcwd()

'C:\\Users\\ajean'